In [1]:
import mysql.connector

def getConnection():
    # step1: connect to MySQL Server
    # localhost = 127.0.0.1
    connection = mysql.connector.connect(host = "127.0.0.1", user = "root", password = "NehalBandal@84", database = "fifa18")
    
    # step2: Perform operation
    return connection


def getFile():
    connection=getConnection()
    cursor=connection.cursor()
    query="SELECT file FROM uploaded_files ORDER BY upload_time DESC LIMIT 1 "
    cursor.execute(query)
    filename=cursor.fetchall()
    return filename

def convertToFloat(df_col):
    result = []
    units = {"K":1000,"M":1000000}
    for val in df_col:
            try:
                result.append( float(val) )  
            except ValueError:
                unit=val[-1]                 
                val = float( val[:-1] )        
                result.append( val * units[unit] )
    
    return result

In [2]:
import numpy as np
import pandas as pd
import os
from unidecode import unidecode
from pandas.api.types import is_string_dtype
import joblib

In [17]:
# name=getFile()
#print(name)
# Step 1 : Read the dataset

# readFile = pd.read_csv(name[0][0], low_memory=False)
# path = os.path.join(os.path.dirname(__file__), "upload/")
readFile = pd.read_csv(r"E:\Projects\fifa18-cdac\upload\sample_5.csv")

columns=['ID', 'Name', 'Age', 'Nationality', 'Potential', 'Club',
       'Value', 'Wage', 'Height_cm', 'Weight_kg', 'Acceleration', 'Aggression',
       'Agility', 'Balance', 'Ball control', 'Composure', 'Crossing', 'Curve',
       'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving',
       'GK handling', 'GK kicking', 'GK positioning', 'GK reflexes',
       'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing',
       'Long shots', 'Marking', 'Penalties', 'Positioning', 'Reactions',
       'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed',
       'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys']

attrs=['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure',
   'Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy',
   'GK diving', 'GK handling', 'GK kicking', 'GK positioning',
   'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing', 'Long shots',
	'Marking', 'Penalties', 'Positioning', 'Reactions', 'Short passing', 'Shot power', 
   'Sliding tackle', 'Sprint speed',	'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys'] 

df=pd.DataFrame(readFile,columns=columns)
print("--------------------Step 1 DONE !!! ---------------------------")

--------------------Step 1 DONE !!! ---------------------------


In [18]:
df.head()

,ID,Name,Age,Nationality,Potential,Club,Value,Wage,Height_cm,Weight_kg,...,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys
0,229960,C. Melody,20,Republic of Ireland,63,Galway United,€90K,€1K,180,75,...,35,47,43,33,68,57,24,37,49,46
1,232520,D. Jaiyesimi,19,England,66,Grimsby Town,€110K,€2K,176,71,...,47,47,50,28,75,40,26,56,46,38
2,234057,H. Al Khulaif,21,Saudi Arabia,61,Al Fateh,€70K,€2K,176,64,...,49,45,21,54,39,56,55,64,40,24
3,240202,K. Ekman,19,Sweden,68,Jönköpings Södra IF,€110K,€1K,175,73,...,52,38,51,13,64,56,12,52,47,39
4,231499,H. Kristiansen,18,Norway,66,Lillestrøm SK,€100K,€1K,190,79,...,49,60,56,48,57,63,45,59,48,37


In [19]:
# Step 2 : Convert accented characters to normal form.

df['Name'] = df['Name'].apply(lambda x:unidecode(str(x)))
df['Club'] = df['Club'].apply(lambda x:unidecode(str(x)))

df["Value"] = df["Value"].str.split('€').str.get(1)
df["Value"]=convertToFloat(df["Value"])
df["Wage"] = df["Wage"].str.split('€').str.get(1)
df["Wage"]=convertToFloat(df["Wage"])

print("--------------------Step 2 DONE !!! ---------------------------")

--------------------Step 2 DONE !!! ---------------------------


In [20]:
nan_cols = df.isna().sum()      # get columns which contain NaN values along with count
nan_cols = nan_cols.to_dict()
nan_cols = [key for key, value in nan_cols.items() if value!=0]
for col in nan_cols:
    df[col] = df[col].fillna(df[col].mean())

[]

In [22]:
cat=["Name", "Nationality", "Club"]
for var in cat:
    df[var]=df[var].fillna("unknown")

In [23]:
for attr in attrs:
    if is_string_dtype(df[attr]):
        df[attr] = df[attr].apply(lambda x: eval(x))
            
for attr in attrs:
    df[attr] = pd.to_numeric(df[attr])


In [28]:
df_continent = pd.read_csv(r"E:\Projects\fifa18-cdac\ml\Datasets\country_continent.csv")

# Create a dictionary {Country: Continent}
newdict = {}
for value in list(range(len(df_continent))):
    newdict[df_continent.iloc[value,0]] = df_continent.iloc[value,1]

#assign continent to each player.
df['Continent'] = ""
for each in list(range(len(df))):
    df['Continent'][each] = newdict[df['Nationality'][each]]   

E:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [39]:
pos_attrs=['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure', 'Crossing',
           'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving', 'Heading accuracy', 'Interceptions', 
           'Jumping', 'Long passing', 'Long shots',	'Marking', 'Penalties', 'Positioning', 
           'Reactions', 'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed',	'Stamina',
           'Standing tackle', 'Strength', 'Vision', 'Volleys']

df_attr=pd.DataFrame(df,columns=pos_attrs)
path = r"E:\Projects\fifa18-cdac\ml\models"
fieldPosition = joblib.load(path+'/field_position.pkl')
output_fieldPosition = fieldPosition.predict(df_attr)
df['FieldPositions']=output_fieldPosition

In [40]:
exactPosition = joblib.load(path + '/exact_position.pkl')
output_exactPosition = exactPosition.predict(df_attr)
df['Position']=output_exactPosition 

In [48]:
attrs = ['Age','Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure', 'Crossing',
           'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving', 'GK handling',
           'GK kicking', 'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions', 
           'Jumping', 'Long passing', 'Long shots',	'Marking', 'Penalties', 'Positioning', 
           'Reactions', 'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed',	'Stamina',
           'Standing tackle', 'Strength', 'Vision', 'Volleys']

In [55]:
df_attr=pd.DataFrame(df,columns=attrs)    
output_col=['ST', 'CF', 'RF', 'LF', 'RW', 'LW', 'RS', 'LS','CAM', 'RAM', 'LAM', 'CM', 'RCM', 'LCM', 'LM', 'RM', 'CDM', 'RDM', 
            'LDM','CB', 'RB', 'LB', 'RCB', 'LCB', 'RWB', 'LWB']

for pos in output_col:
    name=path+"/"+pos+".pkl"
    position_wise_overall = joblib.load(name)
    output_position_wise_overall = position_wise_overall.predict(df_attr)
    df[pos]=output_position_wise_overall
    
df[output_col] = df[output_col].astype('int')

[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [56]:
df.head()

,ID,Name,Age,Nationality,Potential,Club,Value,Wage,Height_cm,Weight_kg,...,CDM,RDM,LDM,CB,RB,LB,RCB,LCB,RWB,LWB
0,229960,C. Melody,20,Republic of Ireland,63,Galway United,90000.0,1000.0,180,75,...,38,38,38,36,41,41,36,36,43,43
1,232520,D. Jaiyesimi,19,England,66,Grimsby Town,110000.0,2000.0,176,71,...,37,37,37,35,40,40,35,35,41,41
2,234057,H. Al Khulaif,21,Saudi Arabia,61,Al Fateh,70000.0,2000.0,176,64,...,46,46,46,51,46,46,51,51,45,45
3,240202,K. Ekman,19,Sweden,68,Jonkopings Sodra IF,110000.0,1000.0,175,73,...,30,30,30,29,32,32,29,29,33,33
4,231499,H. Kristiansen,18,Norway,66,Lillestrom SK,100000.0,1000.0,190,79,...,49,49,49,46,49,49,46,46,49,49


In [57]:
attack = [ 'ST', 'CF', 'RF', 'LF', 'RW', 'LW', 'RS', 'LS' ] 
midfield = [ 'CAM', 'RAM', 'LAM', 'CM', 'RCM', 'LCM', 'LM', 'RM', 'CDM', 'RDM', 'LDM' ]
defense = [ 'CB', 'RB', 'LB', 'RCB', 'LCB', 'RWB', 'LWB' ]

df['Atk']=0
df['Mid']=0
df['Def']=0
    
for i in list(range(0,len(df))):
    atk=df[attack].iloc[i].mean()
    mid=df[midfield].iloc[i].mean()
    defn=df[defense].iloc[i].mean()
    df['Atk'][i]=atk
    df['Mid'][i]=mid
    df['Def'][i]=defn 

E:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
E:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
E:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [61]:
overall = joblib.load(path+'/overall.pkl')
output_overall = overall.predict(df_attr)
df['Overall']=output_overall
df['Overall'] = df['Overall'].astype('int')

In [62]:
df.head()

,ID,Name,Age,Nationality,Potential,Club,Value,Wage,Height_cm,Weight_kg,...,RB,LB,RCB,LCB,RWB,LWB,Atk,Mid,Def,Overall
0,229960,C. Melody,20,Republic of Ireland,63,Galway United,90000.0,1000.0,180,75,...,41,41,36,36,43,43,48,44,39,51
1,232520,D. Jaiyesimi,19,England,66,Grimsby Town,110000.0,2000.0,176,71,...,40,40,35,35,41,41,49,45,38,52
2,234057,H. Al Khulaif,21,Saudi Arabia,61,Al Fateh,70000.0,2000.0,176,64,...,46,46,51,51,45,45,32,38,47,52
3,240202,K. Ekman,19,Sweden,68,Jonkopings Sodra IF,110000.0,1000.0,175,73,...,32,32,29,29,33,33,48,40,31,52
4,231499,H. Kristiansen,18,Norway,66,Lillestrom SK,100000.0,1000.0,190,79,...,49,49,46,46,49,49,47,50,47,51


In [65]:
from sqlalchemy import create_engine
engine = create_engine('mysql+mysqlconnector://root:NehalBandal@84@localhost:3306/unittest', echo=False)

In [66]:
personal = ['ID','Name','Age','Nationality','Continent','Club','Wage','Value']
df_personal=pd.DataFrame(df,columns=personal)
df_personal.to_sql(name='player_personal_info', con=engine, if_exists = 'append', index=False)

In [67]:
physical = ['ID','Height_cm', 'Weight_kg', 'Acceleration', 'Aggression', 'Agility',
       'Balance', 'Ball control', 'Composure', 'Crossing', 'Curve',
       'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving',
       'GK handling', 'GK kicking', 'GK positioning', 'GK reflexes',
       'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing',
       'Long shots', 'Marking', 'Penalties', 'Positioning', 'Reactions',
       'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed',
       'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys']
df_physical=pd.DataFrame(df,columns=physical)
df_physical.to_sql(name='player_physical_attrs', con=engine, if_exists = 'append', index=False)

In [68]:
positional = ['ID','CAM', 'CB', 'CDM', 'CF', 'CM', 'ID', 'LAM', 'LB', 'LCB', 'LCM', 'LDM',
       'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB',
       'RCB', 'RCM', 'RDM', 'RF', 'RM', 'RS', 'RW', 'RWB', 'ST']
df_positional=pd.DataFrame(df,columns=positional)
df_positional.to_sql(name='player_positional_attrs', con=engine, if_exists = 'append', index=False)

In [69]:
abstract = ['ID', 'Atk', 'Mid', 'Def', 'Overall', 'Potential',  'FieldPositions', 'Position']
df_abstract=pd.DataFrame(df,columns=abstract)
df_abstract.to_sql(name='player_abstract_info', con=engine, if_exists = 'append', index=False)